## Build an End-to-End System
This puts together the chain of prompts that you saw throughout the course.

In [1]:
#!/usr/bin/env python3
import os
from dotenv import load_dotenv, find_dotenv
import streamlit as st
from openai import OpenAI

# Load environment variables
load_dotenv(find_dotenv())
api_key = os.getenv("OPENAI_API_KEY")

# Create OpenAI client
client = OpenAI(api_key=api_key)

# Chat completion
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! Can you recommend a product?"}
    ]
)

print(response.choices[0].message.content)



Of course! What type of product are you looking for?


### call to Open AI

In [2]:
# Function to get a completion from messages
def get_completion_from_messages(messages, model="gpt-4", temperature=0.7, max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content
# Function to read a string representation of a list of dictionaries

### System of chained prompts for processing the user query

In [3]:
# Function to get all products and categories
def get_products_and_category():
    # Replace with your real product list if needed
    return ["smartx pro phone", "fotosnap camera", "dslr", "tv", "tvs"]

# Function to extract matched products from user input
def find_category_and_product_only(user_input, product_list):
    user_input = user_input.lower()
    matched = [product for product in product_list if product.lower() in user_input]
    return matched if matched else ["No match found"]

# Function to convert a list of matched products into a list format
def read_string_to_list(category_and_product_response):
    # Already a list, so just return it
    return category_and_product_response

# Function to generate a final output string from a list of categories and products
def generate_output_string(category_and_product_list):
    return "\n".join(str(x) for x in category_and_product_list)

# Example usage
if __name__ == "__main__":
    user_input = "I'm interested in a DSLR and a TV"
    product_list = get_products_and_category()
    matched_products = find_category_and_product_only(user_input, product_list)
    product_list_output = read_string_to_list(matched_products)
    final_output = generate_output_string(product_list_output)
    print(final_output)


dslr
tv


In [4]:
# main.py

# Function to get a completion from messages (mocked for demo)
def get_completion_from_messages(messages):
    # Get the latest user message
    user_message = ""
    for msg in reversed(messages):
        if msg['role'] == 'user':
            user_message = msg['content']
            break

    # Simple keyword-based logic
    user_message_lower = user_message.lower()
    if "hi" in user_message_lower or "hello" in user_message_lower:
        return "Hello! How can I assist you today?"
    elif "phone" in user_message_lower:
        return "We offer several phones, including the SmartX Pro and TCL 503. Would you like more details?"
    elif "camera" in user_message_lower:
        return "Our camera selection includes the FotoSnap DSLR and FotoSnap Compact. Interested in specs or pricing?"
    elif "tvs" in user_message_lower or "tv" in user_message_lower:
        return "We have a range of TVs, including TCL and Samsung models. What size are you looking for?"
    elif "thank" in user_message_lower:
        return "You're welcome! If you have any more questions, just ask."
    elif "bye" in user_message_lower:
        return "Goodbye! Have a great day."
    else:
        return "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."


# Main function to process user messages
def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"
    
    # Step 1: Check input to see if it flags the Moderation API or is a prompt injection
    # For demo, we'll mock moderation (replace with real API in production)
    moderation_output = {"flagged": False}
    # response = openai.Moderation.create(input=user_input)
    # moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        print("Step 1: Input flagged by Moderation API.")
        return "Sorry, we cannot process this request."

    if debug: print("Step 1: Input passed moderation check.")
    
    category_and_product_response = find_category_and_product_only(
    user_input, get_products_and_category()
    )
    category_and_product_list = read_string_to_list(category_and_product_response)
   

    if debug: print("Step 2: Extracted list of products.")

    # Step 3: If products are found, look them up
    product_information = generate_output_string(category_and_product_list)
    if debug: print("Step 3: Looked up product information.")

    # Step 4: Answer the user question
    system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug: print("Step 4: Generated response to user question.")
    all_messages = all_messages + messages[1:]

    # Step 5: Put the answer through the Moderation API
    moderation_output = {"flagged": False}
    # response = openai.Moderation.create(input=final_response)
    # moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        if debug: print("Step 5: Response flagged by Moderation API.")
        return "Sorry, we cannot provide this information."

    if debug: print("Step 5: Response passed moderation check.")

    # Step 6: Ask the model if the response answers the initial user query well
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)
    if debug: print("Step 6: Model evaluated the response.")

    # Step 7: If yes, use this answer; if not, say that you will connect the user to a human
    if "Y" in evaluation_response:  # Using "in" instead of "==" to be safer for model output variation (e.g., "Y." or "Yes")
        if debug: print("Step 7: Model approved the response.")
        return final_response, all_messages
    else:
        if debug: print("Step 7: Model disapproved the response.")
        neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        return neg_str, all_messages

user_input = "tell me about the smartx pro phone and the fotosnap camera, the dslr one. Also what tell me about your tvs"
response, _ = process_user_message(user_input, [])
print(response)

Step 1: Input passed moderation check.
Step 2: Extracted list of products.
Step 3: Looked up product information.
Step 4: Generated response to user question.
Step 5: Response passed moderation check.
Step 6: Model evaluated the response.
Step 7: Model disapproved the response.
I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance.


### Function that collects user and assistant messages over time

In [5]:
# Example of integrating with a simple UI (e.g., Jupyter notebook with ipywidgets)
import ipywidgets as widgets
from IPython.display import display

inp = widgets.Text(
    value='',
    placeholder='Type your message here',
    description='You:',
    disabled=False
)

send_button = widgets.Button(description="Send")

context = []

def collect_messages(debug=False):
    global inp, context
    user_input = inp.value
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''  # ✅ Clears the field

    response, context = process_user_message(user_input, context, debug=True)
    context.append({'role': 'assistant', 'content': f"{response}"})

send_button.on_click(lambda b: collect_messages(debug=True))

display(inp, send_button)

Text(value='', description='You:', placeholder='Type your message here')

Button(description='Send', style=ButtonStyle())

### Chat with the chatbot!¶
Note that the system message includes detailed instructions about what the OrderBot should do.

In [6]:
# filename: chatbot.py


st.set_page_config(page_title="Chat with the Chatbot!", page_icon="🤖")

st.title("Chat with the Chatbot! 🤖")
st.markdown("*(System message includes detailed instructions about OrderBot)*")

# Initialize chat history in session state
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": "You are Service Assistant"}
    ]

# User input
user_input = st.text_input("You:", key="user_input", placeholder="Enter text here...")

# On send button click
if st.button("Send", key="send_button") and user_input:
    st.session_state.messages.append({"role": "user", "content": user_input})

    # Real bot response using your assistant logic
    response, updated_messages = process_user_message(user_input, st.session_state.messages[1:], debug=False)
    st.session_state.messages = [{"role": "system", "content": "You are Service Assistant"}] + updated_messages

    st.rerun()


    # Clear input
    st.experimental_rerun()

# Display chat history
for msg in st.session_state.messages[1:]:  # skip system message
    if msg["role"] == "user":
        st.markdown(f"**You:** {msg['content']}")
    else:
        st.markdown(f"**Service Assistant:** {msg['content']}")



2025-09-23 13:29:57.465 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-23 13:29:57.467 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-23 13:29:57.722 
  command:

    streamlit run /home/oliver/.venv/lib64/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-09-23 13:29:57.724 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-23 13:29:57.726 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-23 13:29:57.727 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-23 13:29:57.728 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored wh